In [ ]:
import os
import subprocess
import sys

# --- CONFIGURAZIONE ---
# Sostituisci questa URL con quella del tuo repository GitHub
REPO_URL = "https://github.com/Biobay/DeepLearning"
# Il nome della cartella del progetto, solitamente il nome del repository
PROJECT_NAME = "DeepLearning" 
# --- FINE CONFIGURAZIONE ---

def run_command(command):
    """Esegue un comando di shell e controlla se ha avuto successo."""
    try:
        print(f"--- Esecuzione: {' '.join(command)} ---")
        # Usiamo sys.executable per essere sicuri di usare il python/pip corretto
        subprocess.run(command, check=True, shell=sys.platform == "win32")
        print("--- Comando completato con successo ---\n")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError) as e:
        print(f"!!! Errore durante l'esecuzione del comando: {e} !!!\n")
        return False

def main():
    """
    Script principale per configurare l'ambiente e avviare il training.
    """
    # 1. Clonare il repository
    if not os.path.exists(PROJECT_NAME):
        print(f"La cartella '{PROJECT_NAME}' non esiste. Clonazione del repository...")
        if not run_command(["git", "clone", REPO_URL]):
            print("Impossibile clonare il repository. Controlla l'URL e che Git sia installato.")
            sys.exit(1)
    else:
        print(f"La cartella '{PROJECT_NAME}' esiste già. Salto la clonazione.")

    # Cambia la directory di lavoro alla cartella del progetto
    try:
        os.chdir(PROJECT_NAME)
        print(f"Spostato nella directory: {os.getcwd()}")
    except FileNotFoundError:
        print(f"Errore: impossibile trovare la directory '{PROJECT_NAME}'.")
        sys.exit(1)

    # 2. Installare le dipendenze
    requirements_file = "requirements.txt"
    if os.path.exists(requirements_file):
        print("Installazione delle dipendenze da requirements.txt...")
        # sys.executable è il percorso dell'interprete Python in uso
        # Usare [sys.executable, "-m", "pip", ...] è un modo robusto per chiamare pip
        if not run_command([sys.executable, "-m", "pip", "install", "-r", requirements_file]):
            print("Impossibile installare le dipendenze. Assicurati che pip sia funzionante.")
            sys.exit(1)
    else:
        print(f"Attenzione: file '{requirements_file}' non trovato. Salto l'installazione delle dipendenze.")

    # 3. Avviare lo script di addestramento
    train_script = os.path.join("scripts", "train.py")
    if os.path.exists(train_script):
        print("Avvio dello script di addestramento...")
        if not run_command([sys.executable, train_script]):
            print("Errore durante l'avvio dello script di addestramento.")
            sys.exit(1)
    else:
        print(f"Errore: script di addestramento non trovato in '{train_script}'.")
        sys.exit(1)

    print("\nProcesso completato!")

if __name__ == "__main__":
    main()